## Project Title

### Project Description

Our team will focus on studying the relationship (if any) of the average land temperature (i.e., #2 proposed dataset) and the evolution of population in such land, along with its GDP. Datasets will be pulled from the World Bank and are available in `.csv`, `.xml` and `.xlsx` formats in the following links, respectively:

[Population](https://data.worldbank.org/indicator/SP.POP.TOTL)

[GDP (2015 USD)](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD)

Behavior will be analysed mainly by visual means (such as scatterplots, pyramid populations, etc.) and reported in Markdown in the corresponding .ipynb file. 

> Imported libraries

In [32]:
import pandas as pd # For data manipulation
import geopandas # For map visualization
import folium # For map visualization
import pycountry # For inconsistent country names

> Previous calculations

Create a dictionary of inconsistent country names and their standardized names:

In [33]:
country_dict = {}
for country in pycountry.countries:
    country_dict[country.name] = country.name
    country_dict[country.alpha_2] = country.name
    country_dict[country.alpha_3] = country.name

---

## Programming Component


### Reading and cleaning the main dataset


In [21]:
df = pd.read_csv('..\data\land-data\GlobalLandTemperaturesByCountry.csv') # Reading GlobalLandTemperaturesByCountry csv file
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


Next setp will be convert `dt` colum in to a date type one and we will only be focusing in the `year` information. 

In [22]:
df.dt = pd.to_datetime(df.dt) # Convert to datetime
df.dt = df.dt.dt.year # Extract year

In [23]:
df.rename(columns={"dt": "Year"}, inplace=True) # Rename column
df_year_temp =df.groupby(['Country', 'Year']).mean() # Group by country and year
df_year_temp.drop(columns=['AverageTemperatureUncertainty'], inplace=True) # Drop unnecessary column
df_year_temp.head() 

AverageTemperature
Country     Year                    
Afghanistan 1838           18.379571
            1839                 NaN
            1840           13.413455
            1841           13.997600
            1842           15.154667

In [24]:
df_year_temp.reset_index(inplace=True) # Let's reset the index so that we can use the Country and Year columns as a key to merge with the other datasets
df_year_temp.head()

,Country,Year,AverageTemperature
0,Afghanistan,1838,18.379571
1,Afghanistan,1839,NaN
2,Afghanistan,1840,13.413455
3,Afghanistan,1841,13.997600
4,Afghanistan,1842,15.154667


---

### Reading and cleaning the 2 others data sets

#### GDP Data

In [27]:
df_gdp = pd.read_csv('..\data\gdp-data\GDP_Country.csv', skiprows=4) # Reading GDP_Country csv file
df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09,NaN
1,Africa Eastern and Southern,AFE,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.536964e+11,1.540611e+11,1.663621e+11,1.749528e+11,1.829721e+11,1.927209e+11,...,8.623341e+11,8.971645e+11,9.231439e+11,9.460928e+11,9.710653e+11,9.964178e+11,1.016728e+12,9.857923e+11,1.029191e+12,NaN
2,Afghanistan,AFG,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.918925e+10,1.971207e+10,1.999816e+10,2.045018e+10,2.099149e+10,2.124113e+10,2.207200e+10,2.155306e+10,1.708357e+10,NaN
3,Africa Western and Central,AFW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.056755e+11,1.076147e+11,1.116749e+11,1.198082e+11,1.262691e+11,1.313913e+11,...,7.046760e+11,7.464664e+11,7.669580e+11,7.678299e+11,7.855332e+11,8.086763e+11,8.344802e+11,8.269667e+11,8.597592e+11,NaN
4,Angola,AGO,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.243377e+10,8.640707e+10,8.721930e+10,8.496904e+10,8.484159e+10,8.372481e+10,8.313874e+10,7.848297e+10,7.934628e+10,NaN


Drop unnecessary columns: 

In [28]:
df_gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 66'], inplace=True)
df_gdp.rename(columns={"Country Name": "Country"}, inplace=True)
df_gdp.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.689383e+09,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09
1,Africa Eastern and Southern,1.536964e+11,1.540611e+11,1.663621e+11,1.749528e+11,1.829721e+11,1.927209e+11,2.002638e+11,2.107883e+11,2.192753e+11,...,8.273424e+11,8.623341e+11,8.971645e+11,9.231439e+11,9.460928e+11,9.710653e+11,9.964178e+11,1.016728e+12,9.857923e+11,1.029191e+12
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.817151e+10,1.918925e+10,1.971207e+10,1.999816e+10,2.045018e+10,2.099149e+10,2.124113e+10,2.207200e+10,2.155306e+10,1.708357e+10
3,Africa Western and Central,1.056755e+11,1.076147e+11,1.116749e+11,1.198082e+11,1.262691e+11,1.313913e+11,1.290167e+11,1.166363e+11,1.183228e+11,...,6.641073e+11,7.046760e+11,7.464664e+11,7.669580e+11,7.678299e+11,7.855332e+11,8.086763e+11,8.344802e+11,8.269667e+11,8.597592e+11
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.854575e+10,8.243377e+10,8.640707e+10,8.721930e+10,8.496904e+10,8.484159e+10,8.372481e+10,8.313874e+10,7.848297e+10,7.934628e+10


In [29]:
df_gdp = df_gdp.melt(id_vars=['Country'], var_name='Year', value_name='GDP') # Melt the dataset to have a single column for the years
df_gdp.head()

,Country,Year,GDP
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,1.536964e+11
2,Afghanistan,1960,NaN
3,Africa Western and Central,1960,1.056755e+11
4,Angola,1960,NaN


In [30]:
df_gdp.sort_values(by=['Country'], inplace=True) # Order by country name
df_gdp.head()

,Country,Year,GDP
2396,Afghanistan,1969,NaN
4790,Afghanistan,1978,NaN
14632,Afghanistan,2015,1.999816e+10
15696,Afghanistan,2019,2.207200e+10
14100,Afghanistan,2013,1.918925e+10


#### Population data

In [35]:
df_pop = pd.read_csv('..\data\population-data\population_data.csv', skiprows=4) # Reading population_data csv file

Drop unnecesary columns:

In [36]:
df_pop.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 66'], inplace=True)
df_pop.rename(columns={"Country Name": "Country"}, inplace=True) # Rename Country column
df_pop.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,59291.0,59522.0,59471.0,...,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0
1,Africa Eastern and Southern,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,153955516.0,158313235.0,162875171.0,...,552530654.0,567891875.0,583650827.0,600008150.0,616377331.0,632746296.0,649756874.0,667242712.0,685112705.0,702976832.0
2,Afghanistan,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,...,30466479.0,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0
3,Africa Western and Central,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,110798486.0,113319950.0,115921723.0,...,376797999.0,387204553.0,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0,478185907.0
4,Angola,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,5868203.0,...,25188292.0,26147002.0,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0


In [38]:
df_pop = df_pop.melt(id_vars=['Country'], var_name='Year', value_name='Population') # Melt the dataset to have a single column for the years
df_pop.head()

,Country,Year,Population
0,Aruba,1960,54608.0
1,Africa Eastern and Southern,1960,130692579.0
2,Afghanistan,1960,8622466.0
3,Africa Western and Central,1960,97256290.0
4,Angola,1960,5357195.0


In [39]:
df_pop.sort_values(by=['Country'], inplace=True) # Order by country name
df_pop.head()

,Country,Year,Population
2396,Afghanistan,1969,10494489.0
4790,Afghanistan,1978,12938862.0
14632,Afghanistan,2015,33753499.0
15696,Afghanistan,2019,37769499.0
14100,Afghanistan,2013,31541209.0


#### Geopandas Dataset

In [40]:
world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))
print(world.head())

       pop_est      continent                      name iso_a3  gdp_md_est  \
0     889953.0        Oceania                      Fiji    FJI        5496   
1   58005463.0         Africa                  Tanzania    TZA       63177   
2     603253.0         Africa                 W. Sahara    ESH         907   
3   37589262.0  North America                    Canada    CAN     1736425   
4  328239523.0  North America  United States of America    USA    21433226   

                                            geometry  
0  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...  
1  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...  
2  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...  
3  MULTIPOLYGON (((-122.84000 49.00000, -122.9742...  
4  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...  



### Combining and cleaning data



### Calculations